## Part 1.1 - Database Management

This first section is for setup and importing data into a new database.

In [2]:
import pandas as pd
import sqlite3

In [3]:
df = pd.read_csv('CarSharing.csv')
df.head()

,id,timestamp,season,holiday,workingday,weather,temp,temp_feel,humidity,windspeed,demand
0,1,2017-01-01 00:00:00,spring,No,No,Clear or partly cloudy,9.84,14.395,81.0,0.0,2.772589
1,2,2017-01-01 01:00:00,spring,No,No,Clear or partly cloudy,9.02,13.635,80.0,0.0,3.688879
2,3,2017-01-01 02:00:00,spring,No,No,Clear or partly cloudy,9.02,13.635,80.0,0.0,3.465736
3,4,2017-01-01 03:00:00,spring,No,No,Clear or partly cloudy,9.84,14.395,75.0,0.0,2.564949
4,5,2017-01-01 04:00:00,spring,No,No,Clear or partly cloudy,9.84,14.395,75.0,0.0,0.000000


In [3]:
conn = sqlite3.connect('CarSharing.db', isolation_level = None)
cur = conn.cursor()

Importing the Carsharing dataframe into a SQLite database.

In [5]:
df.to_sql('CarSharing', conn, if_exists = 'append', index = False)

8708

In [ ]:
#Creating the backup table.

cur.execute("""
CREATE TABLE CarSharingBackup AS
SELECT * FROM CarSharing;
""")

Organizing the temperature data by creating a category column that contains a certain string variable depending on the value of the temperature column.

In [4]:
cur.execute("""
ALTER TABLE CarSharing
ADD COLUMN temp_category;
""")

cur.execute("""
UPDATE CarSharing
SET temp_category = CASE
    WHEN temp_feel < 10 THEN 'Cold'
    WHEN temp_feel > 25 THEN 'Hot'
    ELSE 'Mild'
END;
""")

Now we can move the temperature data to another table and remove these columns from the main table, for the sake of organisation.


In [ ]:
cur.execute("""
CREATE TABLE temperature AS
SELECT temp, temp_feel, temp_category
FROM CarSharing;
""")

To remove the temperature columns from the CarSharing table, I'll temporarily create a new table without these columns and drop the original table.

In [5]:
cur.execute("""
CREATE TABLE NewCarSharing AS
SELECT id, timestamp, season, holiday, workingday, weather, humidity, windspeed, demand, temp_category
FROM CarSharing;
""")

In [6]:
cur.execute("""
DROP TABLE CarSharing;
""")

I'll find the distinct values of 'Weather' in the main table and assign each one an integer code. These codes will be a foreign key to the new 'temperature' table to correspond to the distinct string values.

In [ ]:
cur.execute("""
ALTER TABLE NewCarSharing
ADD COLUMN weather_code
""")

cur.execute("""
SELECT DISTINCT weather
FROM NewCarSharing;
""")

cur.execute("""
UPDATE NewCarSharing
SET weather_code = CASE
    WHEN weather = 'Clear or partly cloudy' THEN '1'
    WHEN weather = 'Mist' THEN '2'
    WHEN weather = 'Light snow or rain' THEN '3'
    ELSE '4'
END;
""")

The weather and weather_code features will be moved to a new 'weather' table.

To remove the weather column, I'll now create the CarSharing table again by selecting every column except the weather column and add it to that table. I will then drop the temporary NewCarSharing table.

In [ ]:
cur.execute("""
CREATE TABLE weather AS 
SELECT DISTINCT weather_code, weather
FROM NewCarSharing;
""")

In [ ]:
cur.execute("""
CREATE TABLE CarSharing AS
SELECT id, timestamp, season, holiday, workingday, humidity, windspeed, demand, temp_category, weather_code
FROM NewCarSharing;
""")

cur.execute("""
DROP TABLE NewCarSharing;
""")

I'll now use strftime() to split the timestamp into hour, day and month columns. This will make the table easier to query when searching between certain dates.

In [ ]:
cur.execute("""
CREATE TABLE time AS
SELECT timestamp, strftime('%H', timestamp) AS 'hour',
strftime('%d', timestamp) AS 'day',
strftime('%m', timestamp) AS 'month'
FROM CarSharing;
""")

With the database appropriately organised, we can use some example queries to find certain pieces of information between dates.

A query to tell which date and time we had the highest demand rate in in 2017:

In [ ]:
cur.execute("""
SELECT timestamp, demand
FROM CarSharing
WHERE timestamp BETWEEN date('2017-01-01') AND date('2017-12-31')
ORDER BY demand DESC;
""")

The highest demand for 2017 was 6.45833828334479 and was recorded at 2017-06-15 17:00:00 (15th June at 5pm).

I can use the new columns created by the strftime() function to query demand rates for each weekday, month and season. We can use a query to find the highest, lowest and average demand rates for each season, weekday and month within a specified timeframe.

In [ ]:
cur.execute("""
CREATE TABLE Q7b AS
SELECT timestamp, season, demand, strftime('%w', timestamp) as 'weekday', strftime('%m', timestamp) as 'month'
FROM CarSharing
WHERE timestamp BETWEEN date('2017-01-01') AND date('2017-12-31');
""")

To make the above table more readable, I'll use a CASE statement to turn the integer values in the 'weekday' and 'month' columns into strings.

In [ ]:
cur.execute("""
ALTER TABLE Q7b
ADD COLUMN weekday_name
""")

cur.execute("""
ALTER TABLE Q7b
ADD COLUMN month_name
""")

cur.execute("""
UPDATE Q7b
SET weekday_name = CASE
    WHEN CAST(weekday as integer) = 0 THEN 'Sunday'
    WHEN CAST(weekday as integer) = 1 THEN 'Monday'
    WHEN CAST(weekday as integer) = 2 THEN 'Tuesday'
    WHEN CAST(weekday as integer) = 3 THEN 'Wednesday'
    WHEN CAST(weekday as integer) = 4 THEN 'Thursday'
    WHEN CAST(weekday as integer) = 5 THEN 'Friday'
    WHEN CAST(weekday as integer) = 6 THEN 'Saturday'
    ELSE NULL
END;
""")

cur.execute("""
UPDATE Q7b
SET month_name = CASE
    WHEN CAST(month AS INTEGER) = 1 THEN 'January'
    WHEN CAST(month AS INTEGER) = 2 THEN 'February'
    WHEN CAST(month AS INTEGER) = 3 THEN 'March'
    WHEN CAST(month AS INTEGER) = 4 THEN 'April'
    WHEN CAST(month AS INTEGER) = 5 THEN 'May'
    WHEN CAST(month AS INTEGER) = 6 THEN 'June'
    WHEN CAST(month AS INTEGER) = 7 THEN 'July'
    WHEN CAST(month AS INTEGER) = 8 THEN 'August'
    WHEN CAST(month AS INTEGER) = 9 THEN 'September'
    WHEN CAST(month AS INTEGER) = 10 THEN 'October'
    WHEN CAST(month AS INTEGER) = 11 THEN 'November'
    WHEN CAST(month AS INTEGER) = 12 THEN 'December'
END;
""")

Demand rates can now be grouped by the season, weekday name or month name in ascending/descending order to easily read the highest and lowest demand rates for that type.

In [ ]:
cur.execute("""
SELECT season, AVG(demand)
FROM Q7b
GROUP BY season
ORDER BY AVG(demand) ASC;
""")

The season with the lowest average demand was Spring, with an average demand value of 3.60685334031046

The season in which demand was highest was Fall, with an average demand value of 4.6602701804789.

In [ ]:
cur.execute("""
SELECT weekday_name, AVG(demand)
FROM Q7b
GROUP BY weekday_name
ORDER BY AVG(demand) ASC;
""")

The weekday with the lowest average demand was Thursdays, with an average demand value of 4.35320177714456.

The weekday with the highest demand was Saturdays, with an average demand value of 4.60194438507599

In [ ]:
cur.execute("""
SELECT month_name, AVG(demand)
FROM Q7_2017
GROUP BY month_name
ORDER BY AVG(demand) ASC;
""")

The month with the lowest average demand was January, with an average demand value of 3.38831232953298.

The month with the highest average demand was July, with an average demand value of 4.78765487641244.

Now that we know which weekday had the highest demand rate, we can break this down further to see the average demand rate at each hour of this weekday throughout a specified timeframe. I'll query within 2017 again:

In [ ]:
#The second query averages the demand values on the given weekday.

cur.execute("""
CREATE TABLE Q7c AS
SELECT timestamp, weekday_name, strftime('%H', timestamp) as 'hour', demand
FROM Q7b;
""")

cur.execute("""
SELECT weekday_name, hour, AVG(demand)
FROM Q7c
WHERE weekday_name IS 'Saturday'
GROUP BY hour
ORDER BY AVG(demand) DESC;
""")

The average demand on Saturdays through 2017 was its highest at 17:00, or 5pm on Saturday, with an average demand value of 5.89513809914992.

The average demand on Saturdays was its lowest at 4:00, or 4am on Saturday, with an average demand of 1.3392704527639.

We can also use queries to explore other features of the database table. We can take a count of the distinct weather conditions defined by the table to see which weather conditions were the most common in a specified timeframe. I can organise this data into new tables.

In [ ]:
cur.execute("""
SELECT timestamp, temp_category, COUNT(temp_category)
FROM CarSharing
WHERE timestamp BETWEEN date('2017-01-01') AND date('2017-12-31')
GROUP BY temp_category;
""")

In 2017, mild weather was the most prevalent at 2735 hours. 

Hot weather was second-most prevalent at 2297 hours.

Cold weather was the rarest in 2017, at only 390 hours recorded at this temperature.

In [ ]:
cur.execute("""
SELECT weather_code, COUNT(weather_code)
FROM CarSharing
GROUP BY weather_code;
""")

‘Clear or partly cloudy’ weather accounted for 5858 hours, 

‘Mist’ accounted for 2142 hours, 

‘Light snow or rain’ accounted for 707 hours and 

‘heavy rain/ice pellets/snow + fog’ accounted for just one hour.

In [ ]:
#Creating the table for windspeed:

cur.execute("""
CREATE TABLE Q7d_windspeed AS
SELECT timestamp, AVG(windspeed), MAX(windspeed), MIN(windspeed), strftime('%Y-%m', timestamp) year_month
FROM CarSharing
WHERE timestamp BETWEEN date('2017-01-01') AND date('2017-12-31')
GROUP BY year_month;
""")

The above table takes the average, maximum and minimum windspeed values for each month in 2017. The values for each month in 2017 are as follows:

Jan 2017 - AVG = 13.7480523584906, MAX = 39.0007, MIN = 0

Feb 2017 - AVG = 15.5777166281755, MAX = 51.9987, MIN = 0

Mar 2017 - AVG = 15.9748841013825, MAX = 40.9973, MIN = 0

Apr 2017 - AVG = 15.8522751121076, MAX = 40.9973, MIN = 0

May 2017 - AVG = 12.4273908277405, MAX = 40.9973, MIN = 0

Jun 2017 - AVG = 11.827618161435, MAX = 35.0008, MIN = 0

Jul 2017 - AVG = 12.0158456570156, MAX = 56.9969, MIN = 0

Aug 2017 - AVG = 12.4111223476298, MAX = 43.0006, MIN = 0

Sep 2017 - AVG = 11.5640800894854, MAX = 40.9973, MIN = 0

Oct 2017 - AVG = 10.8920523702032, MAX = 36.9974, MIN = 0

Nov 2017 - AVG = 12.1422711711712, MAX = 36.9974, MIN = 0

Dec 2017 - AVG = 10.8364595505618, MAX = 43.0006, MIN = 0

In [ ]:
#Creating the table for humidity.

cur.execute("""
CREATE TABLE Q7d_humidity AS
SELECT timestamp, AVG(humidity), MAX(humidity), MIN(humidity), strftime('%Y-%m', timestamp) year_month
FROM CarSharing
WHERE timestamp BETWEEN date('2017-01-01') AND date('2017-12-31')
GROUP BY year_month;
""")

The above table takes the average, maximum and minimum humidity values for each month in 2017. The values for each month in 2017 are as follows:

Jan 2017 - AVG = 56.3076923076923, MAX = 100.0, MIN = 28.0

Feb 2017 - AVG = 53.5807174887892, MAX = 100.0, MIN = 8.0

Mar 2017 - AVG = 55.9977528089888, MAX = 100.0, MIN = 0

Apr 2017 - AVG = 66.2488986784141, MAX = 100.0, MIN = 22.0

May 2017 - AVG = 71.3714285714286, MAX = 100.0, MIN = 24.0

Jun 2017 - AVG = 58.3708609271523, MAX = 100.0, MIN = 20.0

Jul 2017 - AVG = 60.2920353982301, MAX = 94.0, MIN = 17.0

Aug 2017 - AVG = 62.1736263736264, MAX = 94.0, MIN = 25.0

Sep 2017 - AVG = 74.840354767184, MAX = 100.0, MIN = 42.0

Oct 2017 - AVG = 71.5714285714286, MAX = 100.0, MIN = 29.0

Nov 2017 - AVG = 64.1692307692308, MAX = 100.0, MIN = 27.0

Dec 2017 - AVG = 65.1806167400881, MAX = 100.0, MIN = 26.0

In [ ]:
#Creating the table to order average demand by weather condition

cur.execute("""
CREATE TABLE Q7d_demand AS
SELECT AVG(demand), temp_category
FROM CarSharing
GROUP BY temp_category
ORDER BY AVG(demand) DESC;
""")

The above CREATE TABLE statement creates a table that shows the average demand in descending order for each temperature category. The results of that table are as shown:

Hot - Average demand = 4.97154446778135

Mild - Average demand = 4.13086082784128

Cold - Average demand = 3.37549105486628

We can make another table for the month with the highest demand rate, according to the queries defined above. This information can then be compared to other months within the same timeframe.

In [ ]:
# First, I'll find the 2017 month with the highest average demand using a SELECT statement

cur.execute("""
SELECT strftime('%Y-%m', timestamp) year_month, AVG(demand)
FROM CarSharing
WHERE timestamp BETWEEN date('2017-01-01') AND date('2017-12-31')
GROUP BY year_month
ORDER BY AVG(demand) DESC;
""")

The above SELECT statement shows that the month in 2017 with the highest average demand was July, with an average demand value of 4.78765487641244.

In [ ]:
#Compounding all information from (d) into one table for July 2017:

cur.execute("""
CREATE TABLE Q7e AS
SELECT strftime('%Y-%m', timestamp) year_month, AVG(windspeed), MAX(windspeed), MIN(windspeed),  AVG(humidity), MAX(humidity), MIN(humidity)
FROM CarSharing
WHERE year_month IS '2017-07';
""")

July 2017:

Windspeed AVG = 12.0158456570156

Windspeed MAX = 56.9969

Windspeed MIN = 0

Humidity AVG = 60.2920353982301

Humidity MAX = 94.0

Humidity MIN = 17.0